In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from PIL import Image
import os
import base64

# Initialize GPT (vision-capable)
llm = ChatOpenAI(model="gpt-4o-mini")

def ensure_supported_format(image_path: str) -> str:
    """
    Ensures the image is in JPG or PNG format.
    Converts WEBP (or other formats) to JPG automatically.
    Returns the path to the safe file.
    """
    ext = os.path.splitext(image_path)[1].lower()
    if ext in [".jpg", ".jpeg", ".png"]:
        return image_path  # already safe format
    
    # Convert unsupported formats (e.g., .webp) to .jpg
    img = Image.open(image_path).convert("RGB")
    safe_path = image_path.rsplit(".", 1)[0] + ".jpg"
    img.save(safe_path, "JPEG")
    return safe_path

def image_to_base64(image_path: str) -> str:
    """Reads a local image and returns a base64-encoded data URL."""
    with open(image_path, "rb") as f:
        data = f.read()
    b64 = base64.b64encode(data).decode("utf-8")
    ext = os.path.splitext(image_path)[1].lower().replace(".", "")
    if ext == "jpg":
        ext = "jpeg"  # correct MIME type
    return f"data:image/{ext};base64,{b64}"

def describe_image_with_langchain(llm, image_path: str, detail_level: str = "very detailed"):
    # Ensure we have a supported format
    safe_image_path = ensure_supported_format(image_path)
    
    # Encode local file as base64 data URL
    data_url = image_to_base64(safe_image_path)
    
    # Build multimodal input
    message = HumanMessage(
        content=[
            {"type": "text", "text": f"Describe this image in {detail_level} for Midjourney without command, mentioning all visible objects, colors, context, and actions. , no special character like *, -. "},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    )
    
    # Call GPT
    response = llm.invoke([message])
    return response.content

description = describe_image_with_langchain("charmingdesign.webp")
print("Detailed description:\n", description)


Detailed description:
 The image features an inviting winter-themed bedding set, prominently showcasing various decorative pillows and a quilted coverlet, all designed with cheerful snowman motifs. 

1. **Background and Colors**: The primary background is a soft, sky blue, creating a serene winter ambiance. The fabric has a textured appearance, adding depth to the overall composition.

2. **Pillows**: 
   - **Top Left Pillow**: This pillow features two snowmen side by side. The first snowman, on the left, is adorned with a red scarf and a festive red top hat, decorated with a snowflake pattern. This snowman has a joyful expression, complete with a carrot nose and buttons. The second snowman, wearing a blue scarf and hat, is facing slightly away, with a big smile and rosy cheeks, bringing a sense of playfulness.
   - **Top Right Pillow**: A more detailed snowman is depicted, complete with a blue scarf and a red hat. Surrounding him are decorative elements like buttons and snowflakes, en

In [34]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


def improvise_design(llm, base_design: str) -> str:
    """
    Improvises a new design description from a base design using LangChain + LLM.
    """

    # Prompt template
    template = """
    You are a creative design assistant.
    I will give you a base interior design description.
    Please improvise a NEW variation of the design, make it imaginative, detailed, but stay close to the original.
    Return the design only with no command, no special character like *, -. 

    Base design:
    {base_design}

    Now return the improvised design:
    """

    prompt = PromptTemplate(
        input_variables=["base_design"],
        template=template
    )

    # Build chain
    chain = LLMChain(llm=llm, prompt=prompt)

    # Run chain
    improvised_design = chain.run(base_design=base_design)

    return improvised_design.strip()


In [35]:
new_design = improvise_design(llm, description)

In [36]:
new_design

'The image features a charming winter-themed bedding set, adorned with an enchanting array of decorative pillows and a luxurious quilted coverlet, all celebrating whimsical snowman designs.\n\n1. Background and Colors: The backdrop is a gentle, twilight blue, infusing the space with a tranquil wintery glow. The fabric exhibits a subtle shimmer, reminiscent of moonlight on freshly fallen snow, elevating the entire design with an ethereal quality.\n\n2. Pillows:\n   - Top Left Pillow: This pillow showcases two charming snowmen nestled together. The first snowman, on the left, is wrapped in a vibrant emerald green scarf and topped with a jaunty plaid hat, which is adorned with miniature pinecones. His joyous face beams with a friendly smile, featuring a button-studded chest. The second snowman, donning a cozy lavender scarf and a matching beanie, gazes away, displaying a playful smirk, adding a delightful element of surprise.\n   - Top Right Pillow: A strikingly detailed snowman is featur

In [37]:
print(new_design)

The image features a charming winter-themed bedding set, adorned with an enchanting array of decorative pillows and a luxurious quilted coverlet, all celebrating whimsical snowman designs.

1. Background and Colors: The backdrop is a gentle, twilight blue, infusing the space with a tranquil wintery glow. The fabric exhibits a subtle shimmer, reminiscent of moonlight on freshly fallen snow, elevating the entire design with an ethereal quality.

2. Pillows:
   - Top Left Pillow: This pillow showcases two charming snowmen nestled together. The first snowman, on the left, is wrapped in a vibrant emerald green scarf and topped with a jaunty plaid hat, which is adorned with miniature pinecones. His joyous face beams with a friendly smile, featuring a button-studded chest. The second snowman, donning a cozy lavender scarf and a matching beanie, gazes away, displaying a playful smirk, adding a delightful element of surprise.
   - Top Right Pillow: A strikingly detailed snowman is featured, spo